In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [21]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [4]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
sample_submission.columns[1]

'SalePrice'

In [6]:
sample_submission.iloc[:,1].isna().sum()

0

In [28]:
class AutoML: #pour definir une classe
    """
    Classe pour lancer du ML
    """
    
    def __init__(self, nom_df): #constructeur, permet d'instancier  #cible l'objet que l'on est en train de creer
        self.df = pd.read_csv(nom_df)
        
    def preprocessing(self):
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        df_num = train.select_dtypes(include=numerics)
        df_cat = train.select_dtypes(exclude=numerics)

        #transformer les float 64 en float32
        for i in range(df_num.shape[1]):
            if df_num.iloc[:,i].dtypes in ["float64"]:
                df_num.iloc[:,i] = df_num.iloc[:,i].astype("float32")

        for col in df_num.columns:
            if df_num[col].isna().sum()/len(df_num) > 0.20 :
                df_num = df_num.drop(col, 1)
            elif df_num[col].isna().any() == True :
                df_num[col].fillna(df_num[col].mean(), inplace = True)

        for col in df_cat.columns:
            if df_cat[col].isna().sum()/len(df_cat) > 0.20 :
                df_cat = df_cat.drop(col, 1)
            else:
                df_cat[col].dropna(axis=0, how='any')
                dummies = pd.get_dummies(df_cat[col], prefix="col", drop_first= True)
                df_cat = pd.concat([df_cat, dummies], axis=1)  

        df_prepross=pd.concat([df_cat, df_num], axis=1)  
        self.prepross = df_prepross
        
        return df_prepross

    def ML(self,df_prepross,y,n):
        
        #détecter automatiquement si c'est un problème de classification ou de régression
        Y = df_prepross[y]
        X = df_prepross
        
        #splitter les données en train et test
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
        
        #standardiser les données
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        df_num = train.select_dtypes(include=numerics)
        col = df_num.columns
        scaler = preprocessing.StandardScaler()
        X_train[col] = scaler.fit_transform(X_train[col])
        X_test[col] = scaler.fit_transform(X_test[col])
        
        if n == "classification":
        #1 algo de classification
            log_reg = LogisticRegression(max_iter=100)
            log_reg.fit(X_train, y_train)
            y_pred = log_reg.predict(X_train)
        
        elif n == "regression":
        #1 algo de regression
            reg = LinearRegression()
            reg.fit(X_train, y_train)
            y_pred = reg.predict(X_train)
        
        #regarder l'accuracy
        print(accuracy_score(y_train, y_pred))
        
            #return #accuracy et recall
    
    def prediction(self,n):
        #faire la prédiction
        
        return ypred
    
h = AutoML("train.csv")

In [29]:
df_prepross = h.preprocessing()

C:\Users\Alexandra\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [30]:
h.ML(df_prepross, 'SalePrice', "regression")

C:\Users\Alexandra\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Alexandra\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Alexandra\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

ValueError: could not convert string to float: 'RL'

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

train_n = train.select_dtypes(include=numerics)
for col in train_n.columns:
    df.drop(col, axis = 1)

In [ ]:
def binirizer(row):
    if row>50:
        return True
    else:
        return False
    
train_n.apply(binirizer, axis =1)

In [ ]:
h.preprocessing()

In [ ]:
 def preprocessing(self):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df_num = train.select_dtypes(include=numerics)
    df_cat = train.select_dtypes(exclude=numerics)

    #transformer les float 64 en float32
    for i in range(df_num.shape[1]):
        if df_num.iloc[:,i].dtypes in ["float64"]:
            df_num.iloc[:,i] = df_num.iloc[:,i].astype("float32")

    #Retirer les col num avec trop de nans et remplacer le reste avec les moyennes ou drop les lignes
    #df_num = df_num.apply(test_num ,axis=1)
    #df_cat = df_cat.apply(test_cat ,axis=1)
    
    for col in df_num.columns:
        if df_num[col].isna().sum()/len(df_num) > 0.20 :
            df_num = df_num.drop(col, 1)
        elif df_num[col].isna().any() == True :
            df_num[col].fillna(df_num[col].mean(), inplace = True)
    
    for col in df_cat.columns:
        if df_cat[col].isna().sum()/len(df_cat) > 0.20 :
            df_cat = df_cat.drop(col, 1)
        else:
            df_cat[col].dropna(axis=0, how='any')
            dummies = pd.get_dummies(df_cat[col], prefix="col", drop_first= True)
            df_cat = pd.concat([df_cat, dummies], axis=1)  
    
    return df_num, df_cat

In [ ]:
df_num, df_cat = preprocessing(train)

In [ ]:
df_num

In [ ]:
 def preprocessing(train):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df_num = train.select_dtypes(include=numerics)
    df_cat = train.select_dtypes(exclude=numerics)

    #transformer les float 64 en float32
    for i in range(df_num.shape[1]):
        if df_num.iloc[:,i].dtypes in ["float64"]:
            df_num.iloc[:,i] = df_num.iloc[:,i].astype("float32")

    #Retirer les col num avec trop de nans et remplacer le reste avec les moyennes ou drop les lignes
    df_num = test_num(df_num)
    df_cat = test_cat(df_cat)

    return df_num, df_cat

In [ ]:
def test_num(df):
    for col in df.columns:
        if df[col].isna().sum()/len(self.df) > 0.20 :
            df = df.drop(col, 1)
        elif df[col].isna().any() == True :
            df[col].fillna(df[col].mean(), inplace = True)
    return df
            
def test_cat(df):
    for col in df.columns:
        if df[col].isna().sum()/len(self.df) > 0.20 :
            df = df.drop(col, 1)
        else:
            df[col].dropna(axis=0, how='any')
            dummies = pd.get_dummies(df[col], prefix=col, drop_first= True)
            df = pd.concat([df, dummies], axis=1)  

In [ ]:
df_num, df_cat = preprocessing(train)